参考：[GPT-2をファインチューニングしてニュース記事のタイトルを条件付きで生成してみた。 - Qiita](https://qiita.com/m__k/items/36875fedf8ad1842b729)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# ソースコードから直接transformersをインストール
!pip install git+https://github.com/huggingface/transformers
# rinna/japanese-gpt2-mediumのtokenizerはsentencepieceなのでsentencepieceもインストールする必要があります。
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ini1sejs
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ini1sejs
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 7.5 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 70.6 MB/s 
  Created wheel for transformers: filename=transformers-4.20.0.dev0-py3-none-any.whl size=4342386 sha256=bda562407032a3ffef94b4b006d074091fcfdc4431d11a4b8d1906832bdf9fb0
  Stored in directory: /tmp/pip-ephem-wheel-cache-4j1bclcd/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstal

In [4]:
!git clone https://github.com/huggingface/transformers

# run_clm.pyを使います
!ls ./transformers/examples/pytorch/language-modeling/
# README.md   run_clm_no_trainer.py  run_mlm_no_trainer.py  run_plm.py
# requirements.txt  run_clm.py       run_mlm.py

Cloning into 'transformers'...
remote: Enumerating objects: 97414, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 97414 (delta 44), reused 66 (delta 38), pack-reused 97316
Receiving objects: 100% (97414/97414), 90.08 MiB | 26.45 MiB/s, done.
Resolving deltas: 100% (71648/71648), done.
README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True # due to some bug of tokenizer config loading
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/787k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

SPECIAL_TOKENS['bos_token'] + キーワード + SPECIAL_TOKEN['sep_token'] + キーワードを含む文 + SPECIAL_TOKEN['eos_token']

In [6]:
data_path = "drive/MyDrive/ColabNotebooks/data/akutagawa_kumonoito_splitted.txt"
data_joined_path = "drive/MyDrive/ColabNotebooks/data/data_joined.txt"
data_list = []
sent = ""

with open(data_path, 'r', encoding="shift-jis") as f:
  data_list = f.readlines()

with open(data_joined_path, 'w', encoding="utf-8") as f:  
  for data in data_list:
    pieces = data.split()
    sent = "<s>" + pieces[0] + "[SEP]" + "".join(pieces).replace("\n", "") + "</s>"
    f.write(sent + "\n")

In [7]:
print(sent)

<s>極楽[SEP]極楽ももう午に近くなったのでございましょう</s>


In [8]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=drive/MyDrive/ColabNotebooks/data/data_joined.txt \
    --validation_file=drive/MyDrive/ColabNotebooks/data/data_joined.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=5 \
    --save_total_limit=3 \
    --save_steps=5000 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=drive/MyDrive/ColabNotebooks/output_gpt2/ \
    --use_fast_tokenizer=False \
    --overwrite_output_dir

06/04/2022 07:46:20 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/04/2022 07:46:20 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=N

In [9]:
model = AutoModelForCausalLM.from_pretrained("drive/MyDrive/ColabNotebooks/output_gpt2/")

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout)

In [20]:
def conditional_generation(keyword, num_gen=20):
    input_text = '<s>'+keyword+'[SEP]'
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    out = model.generate(input_ids, do_sample=True, top_p=0.95, top_k=40, num_return_sequences=num_gen, max_length=256, bad_words_ids=[[1], [3]])
    print("keyword: "+keyword)
    print("generated:")
    for sent in tokenizer.batch_decode(out):
        sent = sent.split('[SEP]</s>')[1]
        sent = sent.replace('</s>', '')
        print(sent)

In [23]:
conditional_generation("海水")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


keyword: 海水
generated:
は、軟体動物門海綿綱の海水生巻椎亜綱に属する海水藻の一種。 海綿類の中では軟体動物に属する部類。 細胞質の細胞と、非細胞質の細胞からなる細胞質の細胞で構成される。 細胞質細胞は、表面に維管束と呼ばれる繊毛状の構造体を持つ。 細胞構造は、ほぼ球形で、細胞膜は薄く、細胞表面は細胞表面でゼラチン質でできている。 細胞壁は脂質でできており、この脂質の層はコラーゲンを層状に堆積している。 この脂質が細胞外膜となり細胞外膜の外側に水が入る。 細胞膜はゼラチン質でできているため、細胞を包むように水が流れ込んでいる。 細胞質表面にゼラチン質の膜がないものもある。 ゼラチン質は細胞膜を包む層とゼラチン質の層の間の層からなる。 ゼラチン質の細胞層はゼラチン質を包むようにしてゼラチン質を持つ。 ゼラチン質はゼラチン質を包むようにしてゼラチン質を持つ。 このゼラチン質はゼラチン質とゼラチン質の間を通るようにしてゼ
海水を濃縮し、冷却するために使用する薬品。 海水中、炭酸ナトリウム等の炭酸塩を使用する。 海水淡水化剤を参照。
海藻 海藻 (曲) - サザンオールスターズの曲。 海藻 (曲) - コブクロの曲。
海水淡水化システム 海水化 海水の同位体 - 天然に存在する同位体のリスト。
海水浴のレジャーボートとして人気ですが、魚釣りをメインにしている方も多くいます。 魚を釣り上げる時、その魚を釣り上げる為に、様々な仕掛けを作る方がほとんどです。 海水浴のレジャーボートを釣り上げる為に、様々な仕掛けを作る方がほとんどです。 それも、多くの方が、釣り上げている仕掛けから、釣り上げている方がほとんどです。 また、魚釣りをしていると、いろいろな魚が釣れる事に気づくはずです。 その魚を釣り上げる為に、色々な仕掛けを作る方が殆どですが、その他にも様々な工夫をして釣っている方が多いです。 潮の流れや流れを把握して、釣り上げるのですが、それに合わせて、餌を釣ったり、流れにのせます。 魚の食い付きをよくする方法として、魚が泳いでいるときに、潮の流れで魚の餌が動かなくなるようにします。 この時に、魚の餌を捕る為に、餌を動かさない様にする方法もあります。 魚を釣ることが出来るようになれば、釣りの喜びも大きくなり、楽しいものになります。 魚釣りをする為のレジャーボートを